In [1]:
import argparse
import functools
import networkx as nx
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.contrib.eager as eager
tf.enable_eager_execution()
import my

/home/gaiyu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
args = argparse.Namespace()
args.depth = 10
args.device = '/cpu:0'
args.eta = 0
# args.device = '/device:GPU:0' # TODO gpu
args.graph = 'soc-Epinions1'
# args.graph = 'soc-Slashdot0811'
# args.graph = 'soc-Slashdot0902'
args.n_features = 8
args.n_iterations = 1000
args.n_machines = 10
args.radius = 3

In [3]:
class Objective:
    def __init__(self, adj, n_machines, eta):
        adj = sp.sparse.triu(adj)
        self.n_nodes, self.n_edges = adj.shape[0], len(adj.data)
        adj = my.sparse_sp2tf(adj)
        adj = tf.cast(adj, tf.float32)
        e_idx = tf.expand_dims(tf.range(0, self.n_edges, dtype=tf.int64), 1)
        idx0 = tf.concat((tf.expand_dims(adj.indices[:, 0], 1), e_idx), 1)
        idx1 = tf.concat((tf.expand_dims(adj.indices[:, 1], 1), e_idx), 1)
        idx = tf.concat((idx0, idx1), 0)
        self.s = tf.SparseTensor(idx, tf.ones(idx.shape[0]), dense_shape=(self.n_nodes, self.n_edges))
        self.n_machines = n_machines
        self.eta = eta

    def __call__(self, x, training=False):
        # TODO incorporate extent of concentration in objective
        # TODO return expected number of edges per machine

        p = tf.nn.softmax(x)
        
        if training:
            y = tf.multinomial(p, num_samples=1)
            y = tf.squeeze(y)
            y = tf.one_hot(y, self.n_machines)
        else:
            y = p

        z = tf.sparse_tensor_dense_matmul(self.s, y)
        z = tf.minimum(z, 1)

        sum_z = tf.reduce_sum(z)
        r = sum_z / float(self.n_nodes)
        q = (tf.reduce_sum(z, 0) + 1) / sum_z
        b = tf.reduce_sum(q * tf.log(q))

        if training:
            joint = r
#             joint = r + b
#             joint = self.eta * r + (1 - self.eta) * b
            log_p = tf.nn.log_softmax(x)
            objective = joint * tf.reduce_sum(y * log_p) / float(self.n_edges)
            return objective, r, b
        else:
            return r, b

In [4]:
class GNNModule(tf.keras.Model):
    def __init__(self, in_features, out_features, adj, deg, radius, activation):
        super().__init__()
        self.adj, self.deg, self.radius = adj, deg, radius
        new_dense = lambda: tf.keras.layers.Dense(input_shape=(in_features,), units=out_features, use_bias=False)
        self.alpha1, self.alpha2, self.alpha3 = new_dense(), new_dense(), new_dense()
        for i in range(radius):
            setattr(self, 'alpha%d' % (i + 4), new_dense())
        self.beta1, self.beta2, self.beta3 = new_dense(), new_dense(), new_dense()
        for i in range(radius):
            setattr(self, 'beta%d' % (i + 4), new_dense())
        self.bn_alpha, self.bn_beta = tf.keras.layers.BatchNormalization(axis=1), tf.keras.layers.BatchNormalization(axis=1)
        self.activation = activation
    
    def call(self, x, training=False):
        deg = self.deg * x
        u = tf.reduce_mean(x, 1, keepdims=True) + tf.zeros_like(x)
        adj = [tf.sparse_tensor_dense_matmul(self.adj, x)]
        matmul = lambda x, y: tf.sparse_tensor_dense_matmul(y, x)
        for i in range(self.radius - 1):
            adj.append(functools.reduce(matmul, (self.adj,) * 2 ** i, adj[-1]))
        alpha = self.alpha1(x) + self.alpha2(x) + self.alpha3(x) + \
            sum(getattr(self, 'alpha%d' % (i + 4))(a) for i, a in enumerate(adj))
        alpha = self.bn_alpha(self.activation(alpha), training=training)
        beta = self.beta1(x) + self.beta2(x) + self.beta3(x) + \
            sum(getattr(self, 'beta%d' % (i + 4))(a) for i, a in enumerate(adj))
        beta = self.bn_beta(beta, training=training)
        return tf.concat((alpha, beta), 1)

class EdgeDense(tf.keras.Model):
    def __init__(self, in_features, out_features, adj):
        super().__init__()
        adj = sp.sparse.triu(adj)
        self.adj = tf.cast(my.sparse_sp2tf(adj), tf.float32)
        self.out_features = out_features
        for i in range(out_features):
            setattr(self, 'dense%d' % i, tf.keras.layers.Dense(input_shape=(in_features,), units=1))
    
    def call(self, x):
        z_list = []
        for i in range(self.out_features):
            z = getattr(self, 'dense%d' % i)(x)
            u = z * self.adj
            v = tf.transpose(z) * self.adj
            z = u.values + v.values
            z_list.append(tf.reshape(z, (-1, 1)))
        z = tf.concat(z_list, 1)
        return z

class GNN(tf.keras.Model):
    def __init__(self, features, n_machines, adj, radius, activation=tf.keras.activations.relu):
        super().__init__()
        self.dense = EdgeDense(features[-1], n_machines, adj)
        deg = tf.constant(adj.sum(1), dtype=tf.float32)
        adj = tf.cast(my.sparse_sp2tf(adj), tf.float32)
        for i, (m, n) in enumerate(zip(features[:-1], features[1:])):
            setattr(self, 'layer%d' % i, GNNModule(m, n, adj, deg, radius, activation))
        self.n_layers = i + 1
    
    def call(self, x, training=False):
        for i in range(self.n_layers):
            x = getattr(self, 'layer%d' % i)(x, training)
        x = self.dense(x)
        return x

In [5]:
g = my.read_edgelist('data/' + args.graph)
adj = nx.adj_matrix(g)
# x = sp.sparse.random(1000, 1000, 1e-5, data_rvs=lambda shape: np.ones(shape))
# adj = (x + x.transpose()).minimum(1)

In [6]:
with tf.device(args.device):
    objective = Objective(adj, args.n_machines, args.eta)
    gnn = GNN((1,) + (args.n_features,) * args.depth, args.n_machines, adj, args.radius)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)

In [7]:
with tf.device(args.device):
    x = tf.constant(adj.sum(1), dtype=tf.float32)
    x -= tf.reduce_mean(x)
    x /= tf.sqrt(tf.reduce_mean(tf.square(x)))

In [8]:
with tf.device(args.device):
    for i in range(args.n_iterations):
        with eager.GradientTape() as tape:
            z = gnn(x, training=True)
            rslt, r, b = objective(z, training=True)
        gradients = tape.gradient(rslt, gnn.variables)
        optimizer.apply_gradients(zip(gradients, gnn.variables)) # TODO tf.train.get_or_create_global_step
        
        r, b = objective(z)
        print('[iteration %d]%f %f %f' % (i + 1, rslt, r, b))

[iteration 1]-42.563362 2.708159 -2.227542
[iteration 2]-43.194542 2.699521 -2.226448
[iteration 3]-43.571594 2.688580 -2.225018
[iteration 4]-47.360310 2.670804 -2.219677
[iteration 5]-46.743496 2.508573 -2.212607
[iteration 6]-47.016361 2.510904 -2.213331
[iteration 7]-47.463520 2.505195 -2.212242
[iteration 8]-47.988914 2.498802 -2.210998
[iteration 9]-48.473480 2.491981 -2.209648
[iteration 10]-49.076553 2.485011 -2.208218
[iteration 11]-49.545200 2.477817 -2.206748
[iteration 12]-50.135265 2.470615 -2.205253
[iteration 13]-50.892166 2.463284 -2.203674
[iteration 14]-47.806210 2.551860 -2.222705
[iteration 15]-45.498005 2.571074 -2.225294
[iteration 16]-48.157257 2.545171 -2.221474
[iteration 17]-41.379639 2.582226 -2.226843
[iteration 18]-48.828384 2.535567 -2.219660
[iteration 19]-49.128162 2.529846 -2.218615
[iteration 20]-49.580154 2.524021 -2.217482
[iteration 21]-49.994381 2.517664 -2.216252
[iteration 22]-51.461346 2.443587 -2.201334
[iteration 23]-53.978809 2.454893 -2.2048

[iteration 184]-161.413681 1.722023 -1.866414
[iteration 185]-162.082855 1.717688 -1.862392
[iteration 186]-163.031357 1.713348 -1.858332
[iteration 187]-164.051819 1.708998 -1.854194
[iteration 188]-164.912994 1.704646 -1.850022
[iteration 189]-165.518921 1.700287 -1.845815
[iteration 190]-166.780441 1.695918 -1.841537
[iteration 191]-167.483643 1.691538 -1.837224
[iteration 192]-167.838760 1.687149 -1.832848
[iteration 193]-169.046005 1.682753 -1.828408
[iteration 194]-169.486435 1.678351 -1.823952
[iteration 195]-170.531967 1.673942 -1.819413
[iteration 196]-171.455338 1.669527 -1.814837
[iteration 197]-172.222534 1.665104 -1.810211
[iteration 198]-173.125641 1.660673 -1.805510
[iteration 199]-86.256516 1.665199 -1.531325
[iteration 200]-174.479019 1.652021 -1.796261
[iteration 201]-175.926849 1.647779 -1.791649
[iteration 202]-176.352951 1.643505 -1.786953
[iteration 203]-176.882950 1.639197 -1.782191
[iteration 204]-177.969360 1.634874 -1.777344
[iteration 205]-178.633957 1.630519

[iteration 364]-104.852547 1.366365 -0.845699
[iteration 365]-105.990135 1.364704 -0.840123
[iteration 366]-107.152611 1.363135 -0.834782
[iteration 367]-107.919365 1.361646 -0.829682
[iteration 368]-109.028770 1.360237 -0.824814
[iteration 369]-110.065735 1.358906 -0.820182
[iteration 370]-111.627052 1.357645 -0.815757
[iteration 371]-112.376801 1.356447 -0.811503
[iteration 372]-113.050079 1.355310 -0.807419
[iteration 373]-114.386848 1.354231 -0.803501
[iteration 374]-115.134186 1.353206 -0.799743
[iteration 375]-116.253624 1.352229 -0.796141
[iteration 376]-117.416267 1.351301 -0.792692
[iteration 377]-229.739258 1.051787 -0.332067
[iteration 378]-119.285027 1.349622 -0.786307
[iteration 379]-230.283417 1.051551 -0.330916
[iteration 380]-231.241501 1.051343 -0.329906
[iteration 381]-231.494812 1.051037 -0.328407
[iteration 382]-232.319992 1.050678 -0.326667
[iteration 383]-233.159546 1.050411 -0.325530
[iteration 384]-279.906403 1.099426 -0.575611
[iteration 385]-280.833252 1.09768

KeyboardInterrupt: 